In [1]:
import requests # para requisições http
from bs4 import BeautifulSoup 
import json
import pandas as pd

In [2]:
header = {'user-agent': 'Mozilla/5.0'}
requisicao = requests.get('https://pt.wikipedia.org/wiki/Unidades_federativas_do_Brasil', 
                          headers = header)
conteudo = requisicao.text

In [3]:
site = BeautifulSoup(conteudo, 'html.parser')

-----------------------------
Encontra a tabela com os dados desejados

In [4]:
tabela = site.find('table', class_="wikitable sortable")
tabela

<table class="wikitable sortable" style="text-align:center; font-size:85%">
<tbody><tr>
<th>Bandeira
</th>
<th>Unidade federativa
</th>
<th><a href="/wiki/ISO_3166-2:BR" title="ISO 3166-2:BR">Abreviação</a>
</th>
<th><a href="/wiki/Lista_de_capitais_do_Brasil" title="Lista de capitais do Brasil">Sede de governo</a>
</th>
<th><a class="mw-redirect" href="/wiki/Lista_de_estados_do_Brasil_por_%C3%A1rea" title="Lista de estados do Brasil por área">Área</a> (km²)
</th>
<th><a class="mw-redirect" href="/wiki/Lista_de_estados_do_Brasil_por_popula%C3%A7%C3%A3o" title="Lista de estados do Brasil por população">População</a> (<a href="/wiki/2014" title="2014">2014</a>)
</th>
<th>Densidade (<a href="/wiki/2005" title="2005">2005</a>)
</th>
<th><a class="mw-redirect" href="/wiki/Lista_de_estados_do_Brasil_por_PIB" title="Lista de estados do Brasil por PIB">PIB</a> (<a href="/wiki/2015" title="2015">2015</a>)
</th>
<th><a class="mw-redirect" href="/wiki/Lista_de_estados_do_Brasil_por_participa%C3%A

--------------------------
Encontra o corpo da tabela

In [5]:
corpo = tabela.find('tbody')
corpo = corpo.findAll('tr')
corpo

[<tr>
 <th>Bandeira
 </th>
 <th>Unidade federativa
 </th>
 <th><a href="/wiki/ISO_3166-2:BR" title="ISO 3166-2:BR">Abreviação</a>
 </th>
 <th><a href="/wiki/Lista_de_capitais_do_Brasil" title="Lista de capitais do Brasil">Sede de governo</a>
 </th>
 <th><a class="mw-redirect" href="/wiki/Lista_de_estados_do_Brasil_por_%C3%A1rea" title="Lista de estados do Brasil por área">Área</a> (km²)
 </th>
 <th><a class="mw-redirect" href="/wiki/Lista_de_estados_do_Brasil_por_popula%C3%A7%C3%A3o" title="Lista de estados do Brasil por população">População</a> (<a href="/wiki/2014" title="2014">2014</a>)
 </th>
 <th>Densidade (<a href="/wiki/2005" title="2005">2005</a>)
 </th>
 <th><a class="mw-redirect" href="/wiki/Lista_de_estados_do_Brasil_por_PIB" title="Lista de estados do Brasil por PIB">PIB</a> (<a href="/wiki/2015" title="2015">2015</a>)
 </th>
 <th><a class="mw-redirect" href="/wiki/Lista_de_estados_do_Brasil_por_participa%C3%A7%C3%A3o_no_PIB" title="Lista de estados do Brasil por participaç

-------------------------------
Encontra o cabeçalho da tabela que contém a definição dos dados (nome das colunas)


In [6]:
header = corpo[0].findAll('th')
header = [cell.get_text() for cell in header]
print(header)

['Bandeira\n', 'Unidade federativa\n', 'Abreviação\n', 'Sede de governo\n', 'Área (km²)\n', 'População (2014)\n', 'Densidade (2005)\n', 'PIB (2015)\n', '(% total) (2015)\n', 'PIB per capita (R$) (2015)\n', 'IDH (2010)\n', 'Alfabetização (2016)\n', 'Mortalidade infantil (2016)\n', 'Expectativa de vida (2016)\n']


----------------------------------------------------------
- Laço dentro de laço
- Primeiro 'for' viaja pelas linhas, o segundo pelas colunas encontrando o dado
- Verifica se o dado é uma imagem antes 
- Se não for, faz um tratamento rápido para algumas ocasiões especiais, e adiciona o 'dado' na lista 'dados_tabela' que contém todos os dados da tabela
- Se o dado for uma imagem, será adicionado um: "--"

In [7]:
dados_tabela = []

for i in range (len(corpo)):  
    linha = corpo[i].findAll('td')
    dados_tabela.append([])
    
    for j in range (len(linha)):
        dado = linha[j]
        if dado.find('span', class_="mw-image-border") == None:
            dados_tabela[i].append(dado.get_text())
        else:
            dados_tabela[i].append('--')

#LIMPAR PRIMEIRO TERMO VAZIO
del dados_tabela[0]
#TESTES
print(dados_tabela)
print(dados_tabela[26][0])

#ARMAZENA NUMERO DE LINHAS
num_linhas = len(dados_tabela)
num_linhas

[['--', 'Acre\n', 'AC\n', 'Rio Branco\n', '164\xa0122,2\n', '795\xa0145\n', '4,30\n', '13\xa0622\xa0000\n', '0,2\n', '16\xa0953,46\n', '0,663\n', '86,9%\n', '17,0‰\n', '73,9 anos\n'], ['--', 'Alagoas\n', 'AL\n', 'Maceió\n', '27\xa0767,7\n', '3\xa0327\xa0551\n', '108,61\n', '46\xa0364\xa0000\n', '0,8\n', '13\xa0877,53\n', '0,631\n', '80,6%\n', '19,5‰\n', '71,6 anos\n'], ['--', 'Amapá\n', 'AP\n', 'Macapá\n', '142\xa0814,6\n', '756\xa0500\n', '4,16\n', '13\xa0861\xa0000\n', '0,2\n', '18\xa0079,54\n', '0,708\n', '95%\n', '23,2‰\n', '73,9 anos\n'], ['--', 'Amazonas\n', 'AM\n', 'Manaus\n', '1\xa0570\xa0745,7\n', '3\xa0893\xa0763\n', '2,05\n', '86\xa0560\xa0000\n', '1,4\n', '21\xa0978,95\n', '0,674\n', '93,1%\n', '18,2‰\n', '71,9 anos\n'], ['--', 'Bahia\n', 'BA\n', 'Salvador\n', '564\xa0692,7\n', '15\xa0150\xa0143\n', '24,46\n', '245\xa0025\xa0000\n', '4,1\n', '16\xa0115,89\n', '0,660\n', '87%\n', '17,3‰\n', '73,5 anos\n'], ['--', 'Ceará\n', 'CE\n', 'Fortaleza\n', '148\xa0825,6\n', '8\xa0867\

27

------------------------
- Cria um dicionário dos dados
- Adiciona os dados da lista dados_tabela no dicionário_dados

In [8]:
dicionario_dados = {} 
for i in range (len(header)):
    chave = header[i]
    dicionario_dados[chave] = []
    for j in range (num_linhas):
        dicionario_dados[chave].append(dados_tabela[j][i])

#Testes
dicionario_dados.keys()

dict_keys(['Bandeira\n', 'Unidade federativa\n', 'Abreviação\n', 'Sede de governo\n', 'Área (km²)\n', 'População (2014)\n', 'Densidade (2005)\n', 'PIB (2015)\n', '(% total) (2015)\n', 'PIB per capita (R$) (2015)\n', 'IDH (2010)\n', 'Alfabetização (2016)\n', 'Mortalidade infantil (2016)\n', 'Expectativa de vida (2016)\n'])

In [9]:
#TESTES
var_teste = dicionario_dados['Sede de governo'][2]
print('====== PRINT DA VARIAVEL ======== n')
print(var_teste)
print('======== MANIPULAÇÃO =========')
print()
print('========= TYPE ==========')
print(type(var_teste))

KeyError: 'Sede de governo'

In [20]:
dicionario_dados_df = pd.DataFrame(dicionario_dados)

In [21]:
estados_json = json.dumps(dicionario_dados)

In [24]:
with open("dados_wikipedia_estados_da_federacao.json", "w") as arquivo:
    arquivo.write(estados_json)

In [25]:
display(dicionario_dados_df) 

,Bandeira\n,Unidade federativa\n,Abreviação\n,Sede de governo\n,Área (km²)\n,População (2014)\n,Densidade (2005)\n,PIB (2015)\n,(% total) (2015)\n,PIB per capita (R$) (2015)\n,IDH (2010)\n,Alfabetização (2016)\n,Mortalidade infantil (2016)\n,Expectativa de vida (2016)\n
0,--,Acre\n,AC\n,Rio Branco\n,"164 122,2\n",795 145\n,"4,30\n",13 622 000\n,"0,2\n","16 953,46\n","0,663\n","86,9%\n","17,0‰\n","73,9 anos\n"
1,--,Alagoas\n,AL\n,Maceió\n,"27 767,7\n",3 327 551\n,"108,61\n",46 364 000\n,"0,8\n","13 877,53\n","0,631\n","80,6%\n","19,5‰\n","71,6 anos\n"
2,--,Amapá\n,AP\n,Macapá\n,"142 814,6\n",756 500\n,"4,16\n",13 861 000\n,"0,2\n","18 079,54\n","0,708\n",95%\n,"23,2‰\n","73,9 anos\n"
3,--,Amazonas\n,AM\n,Manaus\n,"1 570 745,7\n",3 893 763\n,"2,05\n",86 560 000\n,"1,4\n","21 978,95\n","0,674\n","93,1%\n","18,2‰\n","71,9 anos\n"
4,--,Bahia\n,BA\n,Salvador\n,"564 692,7\n",15 150 143\n,"24,46\n",245 025 000\n,"4,1\n","16 115,89\n","0,660\n",87%\n,"17,3‰\n","73,5 anos\n"
5,--,Ceará\n,CE\n,Fortaleza\n,"148 825,6\n",8 867 448\n,"54,40\n",130 621 000\n,"2,2\n","14 669,14\n","0,682\n","84,8%\n","14,4‰\n","73,8 anos\n"
6,--,Distrito Federal\n,DF\n,Brasília\n,"5 822,1\n",2 867 869\n,"400,73\n",215 613 000\n,"3,6\n","73 971,05\n","0,824\n","97,4%\n","10,5‰\n","78,1 anos\n"
7,--,Espírito Santo\n,ES\n,Vitória\n,"46 077,5\n",3 894 899\n,"73,97\n",120 363 000\n,2\n,"30 627,45\n","0,740\n","93,8%\n","8,8‰\n","78,2 anos\n"
8,--,Goiás\n,GO\n,Goiânia\n,"340 086,7\n",6 551 322\n,"16,52\n",173 632 000\n,"2,9\n","26 265,32\n","0,735\n","93,5%\n","14,9‰\n","74,2 anos\n"
9,--,Maranhão\n,MA\n,São Luís\n,"331 983,3\n",6 861 924\n,"18,38\n",78 475 000\n,"1,3\n","11 366,23\n","0,639\n","83,3%\n","21,3‰\n","70,6 anos\n"


OBS: os dados ainda não foram tratados... Está faltando definir o tipo de cada um. Só pude tratar os IDH e Capital

<h1>Entrar em cada capital e pegar os seus dados!</h1>

In [26]:
#retira o cabeçalho
corpo.pop(0)

<tr>
<td><span class="mw-image-border" typeof="mw:File"><a class="mw-file-description" href="/wiki/Ficheiro:Bandeira_do_Acre.svg"><img class="mw-file-element" data-file-height="350" data-file-width="500" decoding="async" height="32" src="//upload.wikimedia.org/wikipedia/commons/thumb/4/4c/Bandeira_do_Acre.svg/45px-Bandeira_do_Acre.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/4/4c/Bandeira_do_Acre.svg/68px-Bandeira_do_Acre.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/4/4c/Bandeira_do_Acre.svg/90px-Bandeira_do_Acre.svg.png 2x" width="45"/></a></span>
</td>
<td><a href="/wiki/Acre" title="Acre">Acre</a>
</td>
<td>AC
</td>
<td><a href="/wiki/Rio_Branco" title="Rio Branco">Rio Branco</a>
</td>
<td>164 122,2
</td>
<td>795 145
</td>
<td>4,30
</td>
<td>13 622 000
</td>
<td>0,2
</td>
<td>16 953,46
</td>
<td><span style="color: #ffcc00;">0,663</span>
</td>
<td>86,9%
</td>
<td>17,0‰
</td>
<td>73,9 anos
</td></tr>

In [27]:
#acha os links das capitais na quarta coluna (linha[3]) e coloca na lista de urls incompletos
lista_urls = []
for i in range (len(corpo)):  
    linha = corpo[i].findAll('td')
    dado = linha[3]
    dado = dado.find('a')
    dado = dado.get('href')
    lista_urls.append(dado)

lista_urls   

['/wiki/Macei%C3%B3',
 '/wiki/Macap%C3%A1',
 '/wiki/Manaus',
 '/wiki/Salvador',
 '/wiki/Fortaleza',
 '/wiki/Bras%C3%ADlia',
 '/wiki/Vit%C3%B3ria_(Esp%C3%ADrito_Santo)',
 '/wiki/Goi%C3%A2nia',
 '/wiki/S%C3%A3o_Lu%C3%ADs_(Maranh%C3%A3o)',
 '/wiki/Cuiab%C3%A1',
 '/wiki/Campo_Grande_(Mato_Grosso_do_Sul)',
 '/wiki/Belo_Horizonte',
 '/wiki/Bel%C3%A9m_(Par%C3%A1)',
 '/wiki/Jo%C3%A3o_Pessoa',
 '/wiki/Curitiba',
 '/wiki/Recife',
 '/wiki/Teresina',
 '/wiki/Rio_de_Janeiro',
 '/wiki/Natal_(Rio_Grande_do_Norte)',
 '/wiki/Porto_Alegre',
 '/wiki/Porto_Velho',
 '/wiki/Boa_Vista_(Roraima)',
 '/wiki/Florian%C3%B3polis',
 '/wiki/S%C3%A3o_Paulo',
 '/wiki/Aracaju',
 '/wiki/Palmas_(Tocantins)']

------------------------------
<h3>Extrai dados da InfoBox de cada estado de uma maneira não filtrada, mas consegue navegar por todos os links</h3>

In [28]:
infoboxes = []

def extrai_dados_infobox(url_capital):
    #requisição
    header = {'user-agent': 'Mozilla/5.0'}
    requisicao = requests.get(url_capital, headers = header)
    conteudo_capital = requisicao.text
    #pega o html
    site = BeautifulSoup(conteudo_capital, 'html.parser')
    #acha o infobox
    info_box = site.find('table', class_="infobox infobox_v2")
    corpo_info_box = info_box.find('tbody')
    corpo_info_box = corpo_info_box.findAll('td')
    #coloca os dados na lista
   
    return corpo_info_box
    
#completa o link depois navega no site e extrai os dados utilizando a função acima
for i in lista_urls:
    url_completa = 'https://pt.wikipedia.org' + i
    print(url_completa) #quando finalizar uma extração, imprimira o próximo link a ser navegado
    dados_infobox = extrai_dados_infobox(url_completa)
    infoboxes.append(dados_infobox)

https://pt.wikipedia.org/wiki/Macei%C3%B3
https://pt.wikipedia.org/wiki/Macap%C3%A1
https://pt.wikipedia.org/wiki/Manaus
https://pt.wikipedia.org/wiki/Salvador
https://pt.wikipedia.org/wiki/Fortaleza
https://pt.wikipedia.org/wiki/Bras%C3%ADlia
https://pt.wikipedia.org/wiki/Vit%C3%B3ria_(Esp%C3%ADrito_Santo)
https://pt.wikipedia.org/wiki/Goi%C3%A2nia
https://pt.wikipedia.org/wiki/S%C3%A3o_Lu%C3%ADs_(Maranh%C3%A3o)
https://pt.wikipedia.org/wiki/Cuiab%C3%A1
https://pt.wikipedia.org/wiki/Campo_Grande_(Mato_Grosso_do_Sul)
https://pt.wikipedia.org/wiki/Belo_Horizonte
https://pt.wikipedia.org/wiki/Bel%C3%A9m_(Par%C3%A1)
https://pt.wikipedia.org/wiki/Jo%C3%A3o_Pessoa
https://pt.wikipedia.org/wiki/Curitiba
https://pt.wikipedia.org/wiki/Recife
https://pt.wikipedia.org/wiki/Teresina
https://pt.wikipedia.org/wiki/Rio_de_Janeiro
https://pt.wikipedia.org/wiki/Natal_(Rio_Grande_do_Norte)
https://pt.wikipedia.org/wiki/Porto_Alegre
https://pt.wikipedia.org/wiki/Porto_Velho
https://pt.wikipedia.org/wiki

In [29]:
#dados extremamente sujos, mas são de todos os infobox
print(infoboxes)

[[<td style="text-align:center; white-space:normal; font-weight:600; font-size:80%; line-height:105%;"><span class="fn org" style="font-size:130%;">Maceió</span></td>, <td colspan="2" style="vertical-align: top; text-align: center; border-top: 1px solid #a2a9b1;"><figure class="mw-halign-none" typeof="mw:File"><a class="mw-file-description" href="/wiki/Ficheiro:Cidade_de_Macei%C3%B3-AL.jpg" title="Do topo, em sentido horário: vista panorâmica da cidade; Farol da Ponta Verde; Maceió ao pôr do sol; orla de Maceió; vista da cidade à noite e altar da Catedral Metropolitana de Maceió."><img alt="Do topo, em sentido horário: vista panorâmica da cidade; Farol da Ponta Verde; Maceió ao pôr do sol; orla de Maceió; vista da cidade à noite e altar da Catedral Metropolitana de Maceió." class="mw-file-element" data-file-height="1600" data-file-width="1000" decoding="async" height="432" src="//upload.wikimedia.org/wikipedia/commons/thumb/4/46/Cidade_de_Macei%C3%B3-AL.jpg/270px-Cidade_de_Macei%C3%B3-

### Navega pela lista de infoboxes e extrai os dados

In [30]:
dados_cap = []
i= 0

for infobox in infoboxes:
    dados_cap.append([])
    for td in infobox:
        dado = td.get_text()
        dados_cap[i].append(dado)        
    i += 1
print(dados_cap)
    

[['Maceió', 'Do topo, em sentido horário: vista panorâmica da cidade; Farol da Ponta Verde; Maceió ao pôr do sol; orla de Maceió; vista da cidade à noite e altar da Catedral Metropolitana de Maceió.Do topo, em sentido horário: vista panorâmica da cidade; Farol da Ponta Verde; Maceió ao pôr do sol; orla de Maceió; vista da cidade à noite e altar da Catedral Metropolitana de Maceió.\n', '\n Bandeira\nBrasão de armas\n\n', ' Bandeira', 'Brasão de armas', 'Hino\n', 'Gentílico\n', 'maceioense\n', '', 'Localização de Maceió em AlagoasLocalização de Maceió em Alagoas\n', '\nMaceióLocalização de Maceió no Brasil\n', '\nMapa de Maceió', 'Coordenadas\n', '9° 39\' 57" S 35° 44\' 06" O\n', 'País\n', 'Brasil\n', 'Unidade federativa\n', 'Alagoas\n', 'Região metropolitana\n', 'Maceió\n', 'Municípios limítrofes\n', 'Ao Norte: Barra de Santo Antônio, Paripueira e São Luís do Quitunde; Ao Sul: Barra de São Miguel e Marechal Deodoro; Ao Leste: Oceano Atlântico; Ao Oeste: Rio Largo, Satuba, Santa Luzia do

In [35]:
capitais_json = json.dumps(dados_cap)

with open("dados_wikipedia_capitais_da_federacao.json", "w") as arquivo:
    arquivo.write(capitais_json)

In [36]:
df_dados_cap = pd.DataFrame(dados_cap)
display(df_dados_cap)

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,Maceió,"Do topo, em sentido horário: vista panorâmica ...",\n Bandeira\nBrasão de armas\n\n,Bandeira,Brasão de armas,Hino\n,Gentílico\n,maceioense\n,,Localização de Maceió em AlagoasLocalização de...,...,www.maceio.al.gov.br (Prefeitura)www.maceio.al...,None,None,None,None,None,None,None,None,None
1,Macapá,"Do topo, em sentido horário: Marco Zero do Equ...",\n Bandeira\nBrasão de armas\n\n,Bandeira,Brasão de armas,Hino\n,Lema\n,"Macapá, cidade forte\n",Gentílico\n,macapaense[1]\n,...,"R$ 11 735 557,33 mil\n",• Posição\n,BR: 99º\n,PIB per capita (IBGE/2020[7])\n,"R$ 22 880,70\n",Sítio\n,www.macapa.ap.gov.br (Prefeitura)www.macapa.ap...,None,None,None
2,Manaus,,Panorama da região Centro-SulTeatro AmazonasPo...,\n Bandeira\nBrasão de armas\n\n,Bandeira,Brasão de armas,Hino\n,Lema\n,"Grande e livre, radiante e formosa.[1]\n",Gentílico\n,...,PIB per capita (IBGE/2020[7])\n,"R$ 41 345,11\n",Sítio\n,www.manaus.am.gov.br (Prefeitura)www.cmm.am.go...,None,None,None,None,None,None
3,Salvador,,Panorama urbano de Salvador na região da Barra...,\n Bandeira\nBrasão de armas\n\n,Bandeira,Brasão de armas,Hino\n,Lema\n,"Sic illa ad arcam reversa est""Assim ela voltou...",Gentílico\n,...,Gini (PNUD/2010[6])\n,"0,63\n",PIB (IBGE/2020[7])\n,"R$ 58 938 115,10 mil\n",• Posição\n,BA: 1º; NE: 2º\n,PIB per capita (IBGE/2020[7])\n,"R$ 20 417,14\n",Sítio\n,www.salvador.ba.gov.br (Prefeitura)www.cms.ba....
4,Fortaleza,,Panorama urbano de Fortaleza com a Praia de Me...,\n Bandeira\nBrasão de armas\n\n,Bandeira,Brasão de armas,Hino\n,Lema\n,"Fortitudine""Força, fortaleza""\n",Gentílico\n,...,PIB per capita (IBGE/2020[5])\n,"R$ 24 253,93\n",Sítio\n,www.fortaleza.ce.gov.br (Prefeitura)www.cmfor....,None,None,None,None,None,None
5,Brasília,,Eixo Monumental visto a partir da Torre de TVC...,Hino\n,Lema\n,"Venturis ventis(Latim: ""Aos ventos que hão de ...",Gentílico\n,brasiliensecandango\n,,\nBrasíliaLocalização de Brasília no Brasil\n,...,None,None,None,None,None,None,None,None,None,None
6,Vitória,"Do topo, em sentido horário: Catedral Metropol...",\n Bandeira\nBrasão de armas\n\n,Bandeira,Brasão de armas,Hino\n,Lema\n,"Victoria ""Vitória""\n",Gentílico\n,vitoriense\n,...,www.vitoria.es.gov.br (Prefeitura)www.cmv.es.g...,None,None,None,None,None,None,None,None,None
7,Goiânia,,Panorama urbano de GoiâniaMonumento às Três Ra...,\n Bandeira\nBrasão de armas\n\n,Bandeira,Brasão de armas,Hino\n,Lema\n,Pela grandeza da pátria\n,Gentílico\n,...,Sítio\n,www.goiania.go.gov.br (Prefeitura)www.camaragy...,None,None,None,None,None,None,None,None
8,São Luís,\n,\n Bandeira\nBrasão de armas\n\n,Bandeira,Brasão de armas,Hino\n,Gentílico\n,são-luisense ou ludovicense[1]\n,,Localização de São Luís no MaranhãoLocalização...,...,www.saoluis.ma.gov.br (Prefeitura)www.saoluis....,None,None,None,None,None,None,None,None,None
9,Cuiabá,,"Do topo, da esquerda para a direita: vista pan...",\n Bandeira\nBrasão de armas\n\n,Bandeira,Brasão de armas,Hino\n,Gentílico\n,cuiabano\n,,...,• Posição\n,BR: 32º\n,PIB per capita (IBGE/2020[8])\n,"R$ 42 918,31\n",Sítio\n,www.cuiaba.mt.gov.br (Prefeitura)www.camaracba...,None,None,None,None
